In [1]:
import pandas as pd
import numpy as np
import pprint
from nltk.tokenize import TweetTokenizer
#not needed right now (maybe ever?) doing one-hot encoding next cell
#might have to use if we are doing the neutral thing
s = """
#encodes a column of parties into -1, 0, or 1
def encode_party(party_series):
    types = {"Democrat": 0, "Neutral": 0, "Republican": 0}
    output_array = []
    for party in party_series:
        if party == "Democrat":
            types["Democrat"] += 1
        elif party == "Neutral":
            types["Neutral"] += 1
        elif party == "Republican":
            types["Republican"] += 1
            
    resulting_encoding = []

    for type_ in types.keys():
        resulting_encoding.append(type_)
    resulting_encoding.sort()
    return np.asarray(resulting_encoding)
"""

In [2]:
#Hydrating data
tweets = pd.read_csv("data/ExtractedTweets.csv")

#Preprocessing data

#one-hot encoding republican and democrat
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(dtype=np.int, sparse=True, categories='auto')
parties = pd.DataFrame(onehot.fit_transform(tweets[["Party"]])\
                            .toarray(),
                            columns=["Democrat", "Republican"]) #todo add neutral column & training data
parties["Handle"] = tweets.Handle
tweets = tweets.drop(["Party"], axis=1)
encoded_data = pd.merge(parties, tweets)

corpus = []
#Concatinates all tweets in a user's recent tweets, and tokenizes the result
def concatinate_tweets(tweet_object):
    document = " "
    for tweets in tweet_object:
        for single_tweet in tweets.values[0:200]: 
            if not single_tweet.startswith("RT"):
                corpus.append(single_tweet)
                document += single_tweet + " "
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(document)


#dictionary containing the user's handle and a string representation of 200 of their tweets
handles_and_tweets = {}
for handle in encoded_data.groupby(["Handle"])["Tweet"]:
    handles_and_tweets[handle[0]] = concatinate_tweets(handle[1:])



In [3]:
#Organizing the data for training here
df = pd.DataFrame.from_dict(handles_and_tweets, orient='index')
df = df.reset_index()
df.rename(columns={'index':'Handle'}, inplace=True)
finished_product = pd.merge(parties, df, how='right')
finished_product = finished_product.set_index('Handle')
finished_product = finished_product.drop_duplicates()
X = finished_product.iloc[0:, 2:]
Y = finished_product['Democrat']

In [4]:
#tf-idf vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer()
vectors=vectorizer.fit_transform(corpus)

users_tweet_history = pd.Series(X.fillna('').values.tolist()).str.join(' ')

vectorized_tweet_history = vectorizer.transform(users_tweet_history)

tfidf_vectors = pd.DataFrame(vectorized_tweet_history.toarray())
tfidf_vectors

,0,1,2,3,4,5,6,7,8,9,...,109763,109764,109765,109766,109767,109768,109769,109770,109771,109772
0,0.000000,0.015093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.006853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.010251,0.065042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.007036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
429,0.000000,0.007746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
430,0.010073,0.023968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
431,0.007006,0.027785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# W2V encoding
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)


ModuleNotFoundError: No module named 'gensim'

In [ ]:
#word2vec vectorization
user_and_vectorized_tweets_word2vec = {}
k = 0
for index, tweets in X.iterrows(): #todo maybe not all tweets are same length - might just be getting max and assigning that to all of them
    total_embeddings = np.zeros(300) # this number can be tweaked? maybe im not sure
    vectorized_tweet = []
    k+=1
    for word in tweets:
        try:
            encoding_of_word = word_vectors.get_vector(word)
        except KeyError:
            encoding_of_word = np.zeros(300)
        for i, embedding in enumerate(encoding_of_word):
            total_embeddings[i] = total_embeddings[i] + embedding
    for i in range(len(total_embeddings)):
        total_embeddings[i] = total_embeddings[i] / len(tweets)
        
    user_and_vectorized_tweets_word2vec[index] = total_embeddings
    print(f'finished row {k}/433')
    
handle_and_vector_word2vec = pd.DataFrame.from_dict(user_and_vectorized_tweets_word2vec, orient='index')

In [ ]:
#Splitting up the data into train and test
from sklearn.model_selection import train_test_split

#tfidf split
X_train_tfidf, X_test_tfidf, Y_train_tfidf, Y_test_tfidf = train_test_split(tfidf_vectors, Y, test_size = 0.2)

#Word2vec split
X_train_w2v, X_test_w2v, Y_train_w2v, Y_test_w2v = train_test_split(handle_and_vector_word2vec, Y, test_size = 0.2)


In [ ]:
#Training on model - MLP classifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
#Grid search CV
parameters = {'solver': ['lbfgs'],
              'max_iter': [1000,1500,2000 ],
              'alpha': 10.0 ** -np.arange(1, 10),
              'hidden_layer_sizes':np.arange(10, 15)}

tuned_MLP_classifier_w2v = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
tuned_MLP_classifier_tfidf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)


#Word2Vec evaluation
tuned_MLP_classifier_w2v.fit(X_train_w2v, Y_train_w2v)
w2v_score = tuned_MLP_classifier_w2v.score(X_test_w2v.values, Y_test_w2v)
w2v_best_params = tuned_MLP_classifier_w2v.best_params_
print(f'Word2vec score and best params:\n{w2v_score}\n{w2v_best_params}')

#tfidf evaluation
tuned_MLP_classifier_tfidf.fit(X_train_tfidf, Y_train_tfidf)
tfidf_score = tuned_MLP_classifier_tfidf.score(X_test_tfidf.values, Y_test_tfidf)
tfidf_best_params = tuned_MLP_classifier_tfidf.best_params_
print(f'TfIdf score and best params:\n{tfidf_score}\n{tfidf_best_params}')

In [ ]:
#Evaluation for MLP Classifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_predictions_w2v = tuned_MLP_classifier_w2v.predict(X_test_w2v)
print("Word2Vec Confusion matrix and accuracy:")
print(confusion_matrix(Y_test_w2v.astype(int), y_predictions_w2v.astype(int)))
print(accuracy_score(Y_test_w2v.astype(int), y_predictions_w2v.astype(int)))

y_predictions_tfidf = tuned_MLP_classifier_tfidf.predict(X_test_tfidf)
print("\nTfIdf Confusion matrix and accuracy:")
print(confusion_matrix(Y_test_tfidf.astype(int), y_predictions_tfidf.astype(int)))
print(accuracy_score(Y_test_tfidf.astype(int), y_predictions_tfidf.astype(int)))

In [ ]:
#Training on model - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

#Grid search for hyperparameter tuning (for random forest)
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 150, num = 5)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint.pprint(random_grid)
tuned_RF_classifier_w2v = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,
                                  n_iter = 100, cv = 3, verbose=2, random_state=42,
                                  n_jobs = -1)
tuned_RF_classifier_tfidf = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,
                                  n_iter = 100, cv = 3, verbose=2, random_state=42,
                                  n_jobs = -1)
tuned_RF_classifier_w2v.fit(X_train_w2v, Y_train_w2v)
tuned_RF_classifier_tfidf.fit(X_train_tfidf, Y_train_tfidf)

In [ ]:
#Evaluation for RandomForest Classifier

#Evaluation:
print("Word2Vec results:")
print(f'Training Score: {tuned_RF_classifier_w2v.score(X_train_w2v, Y_train_w2v)} '
      + f'\n Validation Score: {tuned_RF_classifier_w2v.score(X_test_w2v, Y_test_w2v)}')

#Evaluation:
print("TfIdf results:")
print(f'Training Score: {tuned_RF_classifier_tfidf.score(X_train_tfidf, Y_train_tfidf)} '
      + f'\n Validation Score: {tuned_RF_classifier_tfidf.score(X_test_tfidf, Y_test_tfidf)}')

In [ ]:
#Compare to data on Congressional Meetings Speech
import glob

passages = []
labels = []
for file in glob.glob("./data/congressional_hearings_data/" +"*D*.txt"):
    inputfile = open(file)
    a = inputfile.readline()
    passages.append(a)
    labels.append(1)
    inputfile.close()

for file in glob.glob("./data/congressional_hearings_data/" +"*R*.txt"):
    inputfile = open(file)
    a = inputfile.readline()
    passages.append(a)
    labels.append(0)
    inputfile.close()


#vectorize sents with tfidf
passages_vectorized = vectorizer.transform(passages)
passages_tfidf = pd.DataFrame(passages_vectorized.toarray())

#vectorize sents with word2vec
word2vec_passages = []
for passage in passages:
    word_list = passage.split(" ")
    for word in word_list:
        total_embeddings = np.zeros(300)
        try:
            encoding_of_word = word_vectors.get_vector(word)
        except KeyError:
            encoding_of_word = np.zeros(300)
        for i, embedding in enumerate(encoding_of_word):
            total_embeddings[i] = total_embeddings[i] + embedding
    for i in range(len(total_embeddings)):
        total_embeddings[i] = total_embeddings[i] / len(word_list)
        
    word2vec_passages.append(total_embeddings)

#Evaluate on models
#TFIDF - MLP
print("TFIDF results:")
passages_tfidf_mlp = tuned_MLP_classifier_tfidf.score(passages_tfidf.values, labels)
print(f'\tMLP score of passages is: {passages_tfidf_mlp}')

#TFIDF - RF
passages_tfidf_rf = tuned_RF_classifier_tfidf.score(passages_tfidf.values, labels)
print(f'\tRF score of passages is: {passages_tfidf_rf}')

#W2V - MLP
print("Word2Vec results:")
passages_w2v_mlp = tuned_MLP_classifier_w2v.score(word2vec_passages, labels)
print(f'\tMLP Score of passages is: {passages_w2v_mlp}')

#W2V - RF
passages_w2v_rf = tuned_RF_classifier_w2v.score(word2vec_passages, labels)
print(f'\tRF score of passages is: {passages_w2v_rf}')

In [ ]:
# Compare to FB data

# populate df 
fb_df = pd.read_csv("data/FacebookPosts.csv")

# one-hot encoding the parties
fb_parties = pd.DataFrame(onehot.fit_transform(fb_df[["Party"]]).toarray(), columns=["Democrat", "Republican"])
fb_parties["Id"] = fb_df.Id
fb_df = fb_df.drop(["Party"], axis=1)
fb_df = pd.merge(fb_parties, fb_df)

# concatenating posts and grouping them by Id
group = fb_df.groupby(['Id'])
list(group['Text'])

def concatenate_posts(posts_object):
    document = " "
    for posts in posts_object:
        for post in posts.values[0:200]: 
            document += post + " "
    # just using the tweet tokenizer because it should tokenize fb posts just fine
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(document)

ids_and_posts = {}
for i in encoded_fb_data.groupby(["Id"])["Text"]:
    ids_and_posts[i[0]] = concatenate_posts(list(i[1:]))

In [ ]:
# organizing the data for testing
#Organizing the data for training here
ids_and_posts_df = pd.DataFrame.from_dict(ids_and_posts, orient='index')
ids_and_posts_df = ids_and_posts_df.reset_index()
ids_and_posts_df.rename(columns={'index':'Id'}, inplace=True)
finished_fb_df = pd.merge(fb_parties, ids_and_posts_df)
finished_fb_df = finished_fb_df.set_index('Id')
finished_fb_df = finished_fb_df.drop_duplicates()
X = finished_fb_df.iloc[0:, 2:]
Y = finished_fb_df['Democrat']

# i think this is right so far but for some reason i had to rerun everything and now it's saying that
# modules cant be found so idk what to do